**Extracts the third phase and returns the same tensor with less samples...**

In [ ]:
import numpy as np
import json, os

# ---------------- CONFIGURATION ----------------
replicate = 2
base_dir = "data/preprocessed"
output_dir = "data/ringdown"

# Input files
tensor_path = os.path.join(base_dir, f"tensor_replicate_{replicate}.npy")
press_path  = os.path.join(base_dir, f"pressures_replicate_{replicate}.json")
nozz_path   = os.path.join(base_dir, f"nozzles_replicate_{replicate}.json")

# Output file
os.makedirs(output_dir, exist_ok=True)
out_tensor_path = os.path.join(output_dir, f"tensor_replicate_{replicate}_increasing_ringdown.npy")
# ------------------------------------------------

# ---------------- LOAD ----------------
if not all(os.path.exists(p) for p in [tensor_path, press_path, nozz_path]):
    raise FileNotFoundError(" Missing one or more preprocessed files. Run the earlier preprocessing step first.")

tensor = np.array(np.load(tensor_path, allow_pickle=True))
with open(press_path, "r") as f:
    pressures = np.array(json.load(f), dtype=float)
with open(nozz_path, "r") as f:
    nozzles = json.load(f)

n_nozzles, n_samples, n_pressures = tensor.shape
print(f" Loaded tensor: {tensor.shape} (nozzles × samples × pressures)")

# ---------------- FIXED CROP ----------------
start_offset = 35  # remove first 35 samples
if start_offset >= n_samples:
    raise ValueError("⚠️ Start offset is larger than waveform length!")

# Compute new length
min_len = n_samples - start_offset
print(f" Cropping first {start_offset} samples → keeping {min_len} samples per waveform")

# Slice the tensor
trimmed_tensor = tensor[:, start_offset:, :]

# ---------------- SAVE ----------------
np.save(out_tensor_path, trimmed_tensor)
print(f" Saved ringdown-only tensor: {out_tensor_path}")
print(f"Shape: {trimmed_tensor.shape} (nozzles × ringdown samples × pressures)")



✅ Loaded tensor: (1280, 124, 16) (nozzles × samples × pressures)
⏱ Cropping first 35 samples → keeping 89 samples per waveform
💾 Saved ringdown-only tensor: data/ringdown\tensor_replicate_2_increasing_ringdown.npy
Shape: (1280, 89, 16) (nozzles × ringdown samples × pressures)


**Visualization for ringdown tensor**

In [ ]:
# --- Visualization: Ringdown-only waveform evolution across pressures ---
import plotly.graph_objects as go
from ipywidgets import interact, FloatRangeSlider
import numpy as np
import json
import os

# ---------------- CONFIGURATION ----------------
replicate = 1           # choose the replicate number
target_nozzle = "A11"  # nozzle to visualize

# Paths
base_dir_pre = "data/preprocessed"  # original metadata (pressures + nozzle list)
base_dir_ring = "data/ringdown"     # new cropped tensor folder

tensor_path = os.path.join(base_dir_ring, f"tensor_replicate_{replicate}_lowering_ringdown.npy")
press_path  = os.path.join(base_dir_pre, f"pressures_replicate_{replicate}_lowering.json")
nozz_path   = os.path.join(base_dir_pre, f"nozzles_replicate_{replicate}_lowering.json")

# ---------------- LOAD ----------------
if not all(os.path.exists(p) for p in [tensor_path, press_path, nozz_path]):
    print(f" Missing ringdown or metadata files for replicate {replicate}. Please check the paths.")
else:
    # Load data
    tensor = np.array(np.load(tensor_path, allow_pickle=True))
    with open(press_path, 'r') as f:
        all_pressures = np.array(json.load(f), dtype=float)
    with open(nozz_path, 'r') as f:
        nozzles = json.load(f)

    print(f"✅ Loaded ringdown tensor for replicate {replicate}:")
    print(f"   Shape = {tensor.shape} (nozzles × samples × pressures)")
    print(f"   {len(nozzles)} nozzles, {len(all_pressures)} pressures.")

    # Check if the target nozzle exists
    if target_nozzle not in nozzles:
        print(f" Nozzle {target_nozzle} not found in replicate {replicate}.")
    else:
        idx = nozzles.index(target_nozzle)

        # --- Range slider for selecting pressure interval ---
        slider = FloatRangeSlider(
            value=[float(all_pressures.min()), float(all_pressures.max())],
            min=float(all_pressures.min()),
            max=float(all_pressures.max()),
            step=0.4,  # adjust to your pressure step
            description='Pressure range (mBar):',
            continuous_update=False,
            layout={'width': '80%'}
        )

        def plot_pressure_range(pressure_range):
            pmin, pmax = pressure_range
            mask = (all_pressures >= pmin) & (all_pressures <= pmax)
            selected_pressures = all_pressures[mask]
            selected_indices = np.where(mask)[0]

            fig = go.Figure()
            for i, p in zip(selected_indices, selected_pressures):
                fig.add_trace(go.Scatter(
                    y=tensor[idx, :, i],
                    x=list(range(tensor.shape[1])),
                    mode='lines',
                    name=f"{p:.1f} mBar",
                    hovertemplate=f"<b>Pressure:</b> {p:.1f} mBar<br>Sample: %{ 'x' }<br>Amplitude: %{ 'y' }<extra></extra>"
                ))

            fig.update_layout(
                title=f"Ringdown waveforms — Nozzle {target_nozzle} (Replicate {replicate})",
                xaxis_title="Sample index (ringdown only)",
                yaxis_title="Amplitude",
                legend=dict(font=dict(size=8), orientation="v", bgcolor="rgba(255,255,255,0.7)"),
                height=500,
                margin=dict(l=40, r=200, t=60, b=40)
            )
            fig.show()

        interact(plot_pressure_range, pressure_range=slider)


✅ Loaded ringdown tensor for replicate 1:
   Shape = (1280, 89, 95) (nozzles × samples × pressures)
   1280 nozzles, 95 pressures.


interactive(children=(FloatRangeSlider(value=(-87.6, -50.0), continuous_update=False, description='Pressure ra…

In [2]:
import numpy as np
import os

# Path to your first batch (adjust if different)
out_dir = r"C:\Users\antho\Downloads"

# Load first batch
params = np.load(os.path.join(out_dir, "params_partial_0_49.npy"))
Q = np.load(os.path.join(out_dir, "Q_partial_0_49.npy"))
rmse = np.load(os.path.join(out_dir, "rmse_partial_0_49.npy"))
nrmse = np.load(os.path.join(out_dir, "nrmse_partial_0_49.npy"))

# Print shapes
print("params:", params.shape)
print("Q:", Q.shape)
print("rmse:", rmse.shape)
print("nrmse:", nrmse.shape)


params: (50, 95, 4)
Q: (50, 95)
rmse: (50, 95)
nrmse: (50, 95)
